<a href="https://colab.research.google.com/github/piantic/Santander-Product-Recommendation-Competition/blob/master/Daecon_Baseline(for_colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Table of Contents

- Data Load
 - Cover time type to datetime
- Missing Values
- Average Score
 - Evaluation

#구글 드라이브 연동

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools

!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null

!apt-get update -qq 2>&1 > /dev/null

!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth

auth.authenticate_user()

from oauth2client.client import GoogleCredentials

creds = GoogleCredentials.get_application_default()

import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL

vcode = getpass.getpass()

!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.13-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
!mkdir -p drive

!google-drive-ocamlfuse drive

##1.1 csv 파일 path 확인

In [0]:
!ls ./drive/DACON/train.csv
trainPath = './drive/DACON/train.csv'
testPath = './drive/DACON/test.csv'
submissionPath = './drive/DACON/submission_1002.csv'

./drive/DACON/train.csv


In [0]:
import pandas as pd
import numpy as np
import pandas_profiling

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import missingno as msno

#Data Load

In [0]:
train = pd.read_csv(trainPath)
test = pd.read_csv(testPath)
submission = pd.read_csv(submissionPath)
print('train shape : ',train.shape)
print('test shape : ',train.shape)
print('submission shape : ',submission.shape)

train shape :  (16909, 1301)
test shape :  (16909, 1301)
submission shape :  (200, 40)


In [0]:
train.head()

,X692,X1272,X553,X1299,X4,X598,X1003,X1010,X1216,X1047,X381,X466,X1420,X686,X711,X1251,X828,X1350,X1066,X1293,X1148,X451,X174,X74,X626,X1083,X1000,X632,X1380,X607,X680,X977,X492,X968,X891,X122,X489,X86,X392,X1007,...,X109,X224,X1259,X1143,X1324,X889,X689,X447,X1270,X827,X1037,X942,X1498,X678,X1100,X239,X1369,X436,X536,X933,X1035,X1194,X1441,X291,X1328,X1323,X616,X1195,X944,X1017,X1164,X656,X1287,X324,X767,X1271,X1288,X58,X1033,X1163
Time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-07-26 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
test.head()

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,X160,X125,X211,X349,X407,X330,X132,X231,X225,X16,X286,X354,X353,X76,X255,X106,X393,X157,X323,X130,X269,X94,X172,X289,X216,X123,X398,X233,X214,X270,...,X377,X232,X333,X140,X44,X37,X42,X422,X50,X386,X80,X250,X142,X235,X446,X331,X102,X186,X243,X194,X183,X139,X459,X457,X70,X368,X282,X426,X47,X192,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-07-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.273,NaN,0.385,NaN,NaN,NaN,NaN,NaN,0.093,NaN,NaN,NaN,0.305,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.860,NaN,NaN,NaN,NaN,NaN,0.469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.275,0.021,NaN
2017-07-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.166,NaN,0.401,NaN,NaN,NaN,NaN,NaN,0.093,NaN,NaN,NaN,0.308,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.792,NaN,NaN,NaN,NaN,NaN,0.477,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.222,0.021,NaN
2017-07-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.164,NaN,0.409,NaN,NaN,NaN,NaN,NaN,0.092,NaN,NaN,NaN,0.344,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.837,NaN,NaN,NaN,NaN,NaN,0.483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.237,0.021,NaN
2017-07-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.168,NaN,0.431,NaN,NaN,NaN,NaN,NaN,0.092,NaN,NaN,NaN,0.312,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.245,NaN,NaN,NaN,NaN,NaN,0.501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.229,0.020,NaN
2017-07-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.196,NaN,0.353,NaN,NaN,NaN,NaN,NaN,0.093,NaN,NaN,NaN,0.306,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.813,NaN,NaN,NaN,NaN,NaN,0.448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.202,0.021,NaN


In [0]:
train.describe()

,X692,X1272,X553,X1299,X4,X598,X1003,X1010,X1216,X1047,X381,X466,X1420,X686,X711,X1251,X828,X1350,X1066,X1293,X1148,X451,X174,X74,X626,X1083,X1000,X632,X1380,X607,X680,X977,X492,X968,X891,X122,X489,X86,X392,X1007,...,X109,X224,X1259,X1143,X1324,X889,X689,X447,X1270,X827,X1037,X942,X1498,X678,X1100,X239,X1369,X436,X536,X933,X1035,X1194,X1441,X291,X1328,X1323,X616,X1195,X944,X1017,X1164,X656,X1287,X324,X767,X1271,X1288,X58,X1033,X1163
count,3064.000000,3065.000000,3041.000000,3006.000000,0.0,3041.000000,2937.000000,3048.000000,3064.000000,3062.000000,6737.000000,6615.000000,3065.000000,3065.000000,3068.000000,3064.000000,3064.000000,3006.000000,3061.000000,3033.000000,3041.000000,6762.000000,6639.000000,3216.000000,3064.000000,3049.000000,3045.000000,3064.000000,3032.000000,3041.000000,3067.000000,3048.000000,3042.000000,3020.000000,2829.000000,16690.000000,3041.000000,16690.000000,6758.000000,3048.000000,...,16512.000000,6671.000000,3066.000000,3016.000000,3034.000000,2828.000000,3064.000000,6895.000000,3065.000000,3038.000000,3063.000000,2829.000000,3043.000000,3039.000000,3032.000000,6772.000000,3006.000000,6758.000000,3042.000000,2860.000000,2475.000000,3014.000000,3065.000000,6868.000000,3034.000000,3032.000000,3067.000000,3040.000000,3112.000000,3045.000000,3040.000000,3066.000000,3035.000000,6766.000000,3040.000000,3065.000000,3034.000000,1747.000000,3044.000000,3041.000000
mean,0.371506,0.410571,0.519877,0.260363,NaN,0.268325,0.378017,0.424614,0.302138,0.273101,0.184522,0.263193,0.456619,0.360088,0.519778,0.384078,0.376258,0.430063,0.410018,0.515061,0.338060,0.335064,0.453441,0.015137,0.219183,0.296045,0.514490,0.408920,0.473640,0.403263,0.508010,0.489797,0.352371,0.604103,0.362605,0.150754,0.297772,1.312882,0.364843,0.280318,...,0.364734,0.354028,0.270500,0.416172,0.405246,0.449531,0.460634,0.336993,0.421908,0.165465,0.552087,0.434308,0.413251,0.422936,0.518576,0.358533,0.410247,0.161904,0.297493,0.435589,0.326013,0.256229,0.320039,0.323304,0.255373,0.474193,0.608112,0.330343,0.682102,0.364462,0.410577,0.441267,0.271726,0.141011,0.384662,0.552852,0.487484,2.881876,0.356959,0.339762
std,0.190358,0.272845,0.236184,0.231955,NaN,0.149223,0.267603,0.267227,0.157183,0.184067,0.106575,0.139884,0.237676,0.298278,0.252190,0.196686,0.297182,0.272137,0.253970,0.278728,0.184448,0.304046,0.221824,0.027044,0.180649,0.186503,0.347605,0.304046,0.242654,0.283495,0.268419,0.277765,0.214490,0.331708,0.233281,0.377574,0.169578,2.323962,0.242012,0.203344,...,0.715089,0.237201,0.142656,0.230763,0.254672,0.246767,0.280525,0.289179,0.347824,0.088152,0.434259,0.239988,0.207303,0.307934,0.299622,0.235787,0.281529,0.161034,0.210053,0.241152,0.370480,0.149899,0.277731,0.177751,0.171673,0.263565,0.461442,0.229693,0.263615,0.302979,0.270049,0.295568,0.146547,0.116648,0.254653,0.386933,0.277884,1.915190,0.189310,0.198999
min,0.049000,0.066000,0.053000,0.017000,NaN,0.034000,0.052000,0.052000,0.071000,0.037000,0.009000,0.024000,0.073000,0.017000,0.037000,0.044000,0.043000,0.068000,0.047000,0.066000,0.035000,0.006000,0.069000,0.001000,0.017000,0.029000,0.059000,0.050000,0.047000,0.029000,0.066000,0.054000,0.049000,0.119000,0.037000,0.003000,0.026000,0.012000,0.029000,0.017000,...,0.003000,0.023000,0.037000,0.080000,0.044000,0.035000,0.036000,0.027000,0.049000,0.010000,0.054000,0.068000,0.053000,0.059000,0.058000,0.023000,0.037000,0.005000,0.027000,0.059000,0.001000,0.026000,0.032000,0.028000,0.020000,0.050000,0.037000,0.036000,0.068000,0.020000,0.058000,0.074000,0.022000,0.007000,0.048000,0.045000,0.041000,0.216000,0.038000,0.057000
25%,0.271000,0.225000,0.416000,0.114000,NaN,0.162000,0.207000,0.285000,0.212000,0.163000,0.100000,0.176000,0.347000,0.150000,0.375000,0.241000,0.191000,0.288000,0.202000,0.344000,0.249000,0.144000,0.345000,0.007000,0.121000,0.202000,0.287000,0.224000,0.389000,0.229000,0.353000,0.288000,0.211000,0.418000,0.200000,0.006000,0.181000,0.078000,0.203000,0.168000,...,0.050000,0.214000,0.194000,0.295000,0.242000,0.288000,0.280000,0.

In [0]:
test.describe()

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,X160,X125,X211,X349,X407,X330,X132,X231,X225,X16,X286,X354,X353,X76,X255,X106,X393,X157,X323,X130,X269,X94,X172,X289,X216,X123,X398,X233,X214,X270,...,X377,X232,X333,X140,X44,X37,X42,X422,X50,X386,X80,X250,X142,X235,X446,X331,X102,X186,X243,X194,X183,X139,X459,X457,X70,X368,X282,X426,X47,X192,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
count,1757.000000,6908.000000,6866.000000,6752.000000,6757.000000,6857.000000,6739.000000,6834.000000,6723.000000,6743.000000,6672.000000,8760.000000,6855.000000,6744.000000,6754.000000,6775.000000,6874.000000,6666.000000,6695.000000,1757.000000,6720.000000,6791.000000,6736.000000,8760.000000,6831.000000,8760.000000,6781.00000,6639.000000,6739.000000,6837.000000,6638.000000,8760.000000,6668.000000,6721.000000,6834.000000,8760.000000,6757.000000,6830.000000,6865.000000,6667.000000,...,6743.000000,6836.00000,6746.000000,6676.000000,1757.000000,1757.000000,1755.000000,6881.000000,1757.000000,6755.000000,8760.000000,6826.000000,6673.000000,6671.000000,6754.000000,6768.000000,8760.000000,6699.000000,6652.000000,6829.000000,6670.000000,6673.000000,6779.000000,6757.000000,8760.000000,6724.000000,6643.000000,6888.000000,1757.000000,6959.000000,6872.000000,7062.000000,6831.000000,6671.000000,1757.000000,6763.000000,6748.000000,8760.000000,8760.000000,6651.000000
mean,1.472014,0.503509,0.339697,0.333051,0.357780,0.226552,0.424325,0.347813,0.096840,0.253864,0.303910,1.200585,0.311255,0.258813,0.431915,0.329086,0.394819,0.208215,0.355569,3.859307,0.401475,0.203394,0.243463,0.814929,0.378135,0.741876,0.33959,0.249717,0.244272,0.463247,0.319149,0.178661,0.120017,0.304175,0.259744,1.242903,0.185374,0.353251,0.440411,0.425522,...,0.134073,0.46239,0.255621,0.407647,2.055121,3.754500,0.375076,0.337295,3.989398,0.351604,7.836023,0.505785,0.508197,0.152455,0.279224,0.285740,1.510764,0.265942,0.367760,0.242667,0.208005,0.398527,0.170325,0.268054,1.189615,0.345386,0.287597,0.316275,0.090738,0.394443,0.230307,0.214052,0.341709,0.323324,0.219365,0.262912,0.237281,1.356384,0.410571,0.374754
std,0.501135,0.344405,0.170168,0.159685,0.262369,0.163054,0.259619,0.161057,0.105429,0.165962,0.143885,2.244679,0.187438,0.277789,0.219129,0.204955,0.268837,0.116085,0.211195,1.379408,0.216582,0.180986,0.156591,1.660453,0.215277,0.642844,0.18194,0.148421,0.136982,0.314075,0.167209,0.481556,0.104749,0.268492,0.256775,2.031765,0.138295,0.416199,0.214907,0.248903,...,0.108066,0.23696,0.180029,0.250110,3.444279,2.013634,0.923620,0.217725,3.081846,0.208671,6.247472,0.328479,0.364999,0.097210,0.166065,0.202702,2.102580,0.292029,0.261123,0.169000,0.125394,0.231289,0.141511,0.202649,2.142232,0.198970,0.236488,0.264300,0.402078,0.261482,0.131057,0.146844,0.184072,0.214725,0.673083,0.169633,0.183557,1.915454,0.845600,0.185928
min,0.712000,0.049000,0.047000,0.038000,0.030000,0.006000,0.051000,0.058000,0.005000,0.015000,0.036000,0.017000,0.021000,0.007000,0.058000,0.036000,0.043000,0.016000,0.021000,1.844000,0.040000,0.018000,0.026000,0.054000,0.005000,0.259000,0.01900,0.027000,0.023000,0.033000,0.033000,0.002000,0.007000,0.018000,0.021000,0.172000,0.016000,0.019000,0.063000,0.030000,...,0.003000,0.05300,0.012000,0.028000,0.027000,1.394000,0.002000,0.023000,0.089000,0.031000,0.257000,0.033000,0.041000,0.010000,0.033000,0.024000,0.267000,0.007000,0.011000,0.019000,0.008000,0.025000,0.007000,0.031000,0.086000,0.027000,0.017000,0.007000,0.060000,0.001000,0.018000,0.004000,0.029000,0.020000,0.088000,0.008000,0.011000,0.113000,0.004000,0.042000
25%,1.098000,0.281000,0.260000,0.245000,0.185000,0.107000,0.276000,0.255000,0.029000,0.152000,0.233000,0.209000,0.195000,0.122000,0.268000,0.174000,0.228000,0.137000,0.192000,2.735000,0.268750,0.108000,0.177000,0.088000,0.253000,0.437000,0.24300,0.148000,0.167000,0.262000,0.214000,0.013000,0.066000,0.149000,0.113000,0.279000,0.092000,0.118000,0.325000,0.277000,...,0.099000,0.33400,0.117000,0.242000,0.029000,2.281000,0.003000,0.216000,1.864000,0.191000,1.621

#Convert time type to datetime

In [0]:
#Convert time type to datetime
train = pd.read_csv(trainPath)

train['Time'] = pd.to_datetime(train.Time)
train = train.set_index('Time')

test = pd.read_csv(testPath)

test['Time'] = pd.to_datetime(test.Time)
test = test.set_index('Time')

In [0]:
train.head()

,X692,X1272,X553,X1299,X4,X598,X1003,X1010,X1216,X1047,X381,X466,X1420,X686,X711,X1251,X828,X1350,X1066,X1293,X1148,X451,X174,X74,X626,X1083,X1000,X632,X1380,X607,X680,X977,X492,X968,X891,X122,X489,X86,X392,X1007,...,X109,X224,X1259,X1143,X1324,X889,X689,X447,X1270,X827,X1037,X942,X1498,X678,X1100,X239,X1369,X436,X536,X933,X1035,X1194,X1441,X291,X1328,X1323,X616,X1195,X944,X1017,X1164,X656,X1287,X324,X767,X1271,X1288,X58,X1033,X1163
Time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-07-26 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#Missing Values

In [0]:
_, ax = plt.subplots(1,2, figsize=(15,5))
sns.distplot(train.isnull().mean(axis=0), ax=ax[0])
ax[0].set_title('Distribution of Missing Values Percentage in Train set')

sns.distplot(test.isnull().mean(axis=0), ax=ax[1])
ax[1].set_title('Distribution of Missing Values Percentage in Test set')
plt.show()

print('train shape : ',train.shape)

train shape :  (16909, 1300)


#Average Score

In [0]:
answer = test.median(axis=0).sort_index()

months = np.zeros((answer.shape[0], 5))
months[:,0] = answer * 30
months[:,1] = answer * 31
months[:,2] = answer * 31
months[:,3] = answer * 30
months[:,4] = answer * 31

avg_submission = submission.copy()
avg_submission = avg_submission.set_index('meter_id')
for i in range(24):
  avg_submission.iloc[:,i] = answer
for i in range(10):
  avg_submission.iloc[:,24+i] = answer * 24
avg_submission.iloc[:,34:] = months

In [0]:
avg_submission.head()

,X2018_7_1_1h,X2018_7_1_2h,X2018_7_1_3h,X2018_7_1_4h,X2018_7_1_5h,X2018_7_1_6h,X2018_7_1_7h,X2018_7_1_8h,X2018_7_1_9h,X2018_7_1_10h,X2018_7_1_11h,X2018_7_1_12h,X2018_7_1_13h,X2018_7_1_14h,X2018_7_1_15h,X2018_7_1_16h,X2018_7_1_17h,X2018_7_1_18h,X2018_7_1_19h,X2018_7_1_20h,X2018_7_1_21h,X2018_7_1_22h,X2018_7_1_23h,X2018_7_1_24h,X2018_7_1_d,X2018_7_2_d,X2018_7_3_d,X2018_7_4_d,X2018_7_5_d,X2018_7_6_d,X2018_7_7_d,X2018_7_8_d,X2018_7_9_d,X2018_7_10_d,X2018_7_m,X2018_8_m,X2018_9_m,X2018_10_m,X2018_11_m
meter_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
X5,5.618,5.618,5.618,5.618,5.618,5.618,5.618,5.618,5.618,5.618,5.618,5.618,5.618,5.618,5.618,5.618,5.618,5.618,5.618,5.618,5.618,5.618,5.618,5.618,134.832,134.832,134.832,134.832,134.832,134.832,134.832,134.832,134.832,134.832,61.17,63.209,63.209,61.17,63.209
X7,1.146,1.146,1.146,1.146,1.146,1.146,1.146,1.146,1.146,1.146,1.146,1.146,1.146,1.146,1.146,1.146,1.146,1.146,1.146,1.146,1.146,1.146,1.146,1.146,27.504,27.504,27.504,27.504,27.504,27.504,27.504,27.504,27.504,27.504,16.02,16.554,16.554,16.02,16.554
X10,2.039,2.039,2.039,2.039,2.039,2.039,2.039,2.039,2.039,2.039,2.039,2.039,2.039,2.039,2.039,2.039,2.039,2.039,2.039,2.039,2.039,2.039,2.039,2.039,48.936,48.936,48.936,48.936,48.936,48.936,48.936,48.936,48.936,48.936,6.93,7.161,7.161,6.93,7.161
X16,3.684,3.684,3.684,3.684,3.684,3.684,3.684,3.684,3.684,3.684,3.684,3.684,3.684,3.684,3.684,3.684,3.684,3.684,3.684,3.684,3.684,3.684,3.684,3.684,88.416,88.416,88.416,88.416,88.416,88.416,88.416,88.416,88.416,88.416,15.51,16.027,16.027,15.51,16.027
X18,2.575,2.575,2.575,2.575,2.575,2.575,2.575,2.575,2.575,2.575,2.575,2.575,2.575,2.575,2.575,2.575,2.575,2.575,2.575,2.575,2.575,2.575,2.575,2.575,61.800,61.800,61.800,61.800,61.800,61.800,61.800,61.800,61.800,61.800,1.86,1.922,1.922,1.86,1.922
